In [38]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='notebook'
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.figure_factory as ff
import sys

In [39]:
freq = 200.0 # Hz, the accelerometer frequency
dt = 1.0/freq
dt_kf = dt*40
acc_steps = int(freq*30+1)
kf_freq = 1.0/dt_kf
kf_steps = int(kf_freq*30+1)
t_acc = np.linspace(0.0, 30.0, acc_steps) # acceleration time vector with dt = 0.02 sec
t_kf = np.linspace(0.0, 30.0, kf_steps) # acceleration time vector with dt = 0.02 sec
omega = 0.1 #rad/sec
a = 10
a_truth = [a * np.sin(omega*t_acc[i]) for i in range(t_acc.shape[0])] # uncorrupted true acceleration values


In [55]:
##### System definition #####
# state transition matrix
Phi = np.array([
    [1, dt_kf, -dt_kf**2/2],
    [0, 1, -dt_kf],
    [0, 0, 1]
])

# Gamma (B) matrix
Gamma = np.array([
    [-dt_kf**2/2],
    [-dt_kf],
    [0]
])

# The process noise variance of the accelerometer with additive white Gaussian noise and zero mean:
w_var = 0.0004 #(meters/sec^2)^2
w_std = np.sqrt(w_var)
w_mean = 0
W = w_var

# H matrix used in z = Hx + eta
H = np.array([
    [1, 0, 0],
    [0, 1, 0]
])

# measurement noise mean and variance
etabar = np.zeros((2,1))
etastd = np.array([[1, 0],[0, 0.04]]) # in units of meter^2 and (meters/sec)^2
etavar = np.dot(etastd, etastd)



##### Mean and variance to use for initializing the Monte Carlo sim #####
# The bias mean and std to draw from
bbar_a = 0
b_var = 0.01 # (meters/sec^2)^2
b0_std = np.sqrt(b_var)

# The mean, std, and variance of the initial position to draw from
Mp_0 = 10**2 # meters^2
p0_std = np.sqrt(Mp_0) # meters, the standard deviation
pbar_0 = 0

# The mean, std, and variance of the initial velocity to draw from
Mv_0 = 1 # (meters/sec)^2
v0_std = np.sqrt(Mv_0) # meters/sec
vbar_0 = 100 # m/s



##### Initial values for KF algo #####
# initial covariance matrix
P0 = np.array([
    [Mp_0, 0, 0],
    [0, Mv_0, 0],
    [0, 0, b_var]
])

# initial position, velocity, bias guesses
vc_0 = vbar_0
pc_0 = pbar_0
ba_0 = 0



print("Phi\n", Phi)
print("Gamma\n", Gamma)
print("Process noise\n", w_var)
print("H matrix\n", H)
print("Sensor Noise\n", etavar)
print("P0\n", P0)
print("b0 std\n", b0_std)


Phi
 [[ 1.    0.2  -0.02]
 [ 0.    1.   -0.2 ]
 [ 0.    0.    1.  ]]
Gamma
 [[-0.02]
 [-0.2 ]
 [ 0.  ]]
Process noise
 0.0004
H matrix
 [[1 0 0]
 [0 1 0]]
Sensor Noise
 [[1.     0.    ]
 [0.     0.0016]]
P0
 [[1.e+02 0.e+00 0.e+00]
 [0.e+00 1.e+00 0.e+00]
 [0.e+00 0.e+00 1.e-02]]
b0 std
 0.1


# The Kalman Filter Algorithm

In [139]:
Mk = np.dot(Phi, np.dot(P0, np.transpose(Phi))) + np.dot(Gamma, np.dot(W, np.transpose(Gamma)))
Pk = np.linalg.inv(np.linalg.inv(Mk) + np.dot(np.transpose(H), np.dot(np.linalg.inv(etavar), H)))
# ikh = np.eye(3) - np.dot(Kk, H)
# Pk = np.dot(ikh, np.dot(Mk, np.transpose(ikh))) + np.dot(Kk, np.dot(etavar, np.transpose(Kk)))
Kk = np.dot(Pk, np.dot(np.transpose(H), np.linalg.inv(etavar)))
count = 1
counts = [count]
Ks = [Kk]
Ms = [Mk]
Ps = [Pk]

while count < 2 or not np.allclose(Ks[-2], Ks[-1], rtol=1.0e-09, atol=1e-09):
    Mk = np.dot(Phi, np.dot(Pk, np.transpose(Phi))) + np.dot(Gamma, np.dot(W, np.transpose(Gamma)))
    Pk = np.linalg.inv(np.linalg.inv(Mk) + np.dot(np.transpose(H), np.dot(np.linalg.inv(etavar), H)))
#     ikh = np.eye(3) - np.dot(Kk, H)
#     Pk = np.dot(ikh, np.dot(Mk, np.transpose(ikh))) + np.dot(Kk, np.dot(etavar, np.transpose(Kk)))
    Kk = np.dot(Pk, np.dot(np.transpose(H), np.linalg.inv(etavar)))
    count+=1
    sys.stdout.write("\rCount %i" % count)
#     sys.stdout.write("K00 %f" %Kk[0,0])
#     sys.stdout.write("\rK01 %f" %Kk[0,1])
#     sys.stdout.write("\rK10 %f" %Kk[1,0])
#     sys.stdout.write("\rK11 %f" %Kk[1,1])
#     sys.stdout.write("\rK20 %f" %Kk[2,0])
#     sys.stdout.write("\rK21 %f" %Kk[2,1])
    sys.stdout.flush()
    counts.append(count)
    Ks.append(Kk.copy())
    Ms.append(Mk.copy())
    Ps.append(Pk.copy())
print("Converged in ", count, " iterations")
print("K")
print(Ks[-1])
print("Mk")
print(Ms[-1])
print("Pk")
print(Ps[-1])

Count 37668Converged in  37668  iterations
K
[[ 7.92042100e-03  1.75727322e-01]
 [ 2.81163715e-04  9.48738003e-02]
 [-3.73316458e-08 -1.25969139e-05]]
Mk
[[ 8.03912007e-03  3.13132051e-04 -4.15762569e-08]
 [ 3.13132051e-04  1.67806569e-04 -2.22805969e-08]
 [-4.15762569e-08 -2.22805969e-08  1.06249963e-08]]
Pk
[[ 7.92042100e-03  2.81163715e-04 -3.73316458e-08]
 [ 2.81163715e-04  1.51798080e-04 -2.01550623e-08]
 [-3.73316458e-08 -2.01550623e-08  1.06247141e-08]]


In [50]:
print(Ks[3])
print(Ks[100])
print(Ks[-100])

[[ 2.49414618e-01  9.17395446e-02]
 [ 1.46783271e-04  5.02695426e-01]
 [-8.72435431e-05 -8.26986168e-01]]
[[ 1.19052394e-02  1.85262764e-01]
 [ 2.96420423e-04  1.05992270e-01]
 [-1.47853386e-05 -5.85546685e-03]]
[[ 7.92042135e-03  1.75727438e-01]
 [ 2.81163901e-04  9.48738633e-02]
 [-3.74300725e-08 -1.26301263e-05]]


count:  2181100
K
[[ 7.92042100e-03  1.75727322e-01]
 [ 2.81163715e-04  9.48738003e-02]
 [-3.73316458e-08 -1.25969139e-05]]
Mk
[[ 8.03912007e-03  3.13132051e-04 -4.15762569e-08]
 [ 3.13132051e-04  1.67806569e-04 -2.22805969e-08]
 [-4.15762569e-08 -2.22805969e-08  1.06249963e-08]]
Pk
[[ 7.92042100e-03  2.81163715e-04 -3.73316458e-08]
 [ 2.81163715e-04  1.51798080e-04 -2.01550623e-08]
 [-3.73316458e-08 -2.01550623e-08  1.06247141e-08]]

# The Monte Carlo Simulation

In [157]:
def monte_carlo_run(verbose = False):
    # Phi, Gamma, W, Etavar, P0, H are already defined
    
    # initialize the true position, velocity, and bias for this Monte Carlo run
    p_0_true = np.random.normal(pbar_0, p0_std)
    v_0_true = np.random.normal(vbar_0, v0_std)
    b_a_true = np.random.normal(bbar_a, b0_std)
    if verbose == True:
        print("p0\n", p_0_true)
        print("v0\n", v_0_true)
        print("b0\n", b_a_true)
    
    # initialize the belief values for the position, velocity, bias, and state vector
    pc_now = pbar_0
    vc_now = vbar_0
    
    # variables to hold the true position and velocity
    ptrue_now = p_0_true
    vtrue_now = v_0_true
    
    ##### A PRIORI #####
    # dxhat_0
    dxhat = np.array([
    [0],
    [0],
    [0]])
    
    # P0
    Pk = np.copy(P0) # this is basically Mk
    if verbose == True:
        print("Pk\n", Pk)
    
    ##### A POSTERIORI #####
    # simulate GPS measurement
    eta_i = np.random.normal(etabar, etastd) # noise on the GPS
    vgps_i = vtrue_now + eta_i[0,0]
    pgps_i = ptrue_now + eta_i[1,1]
    
    
    # initialize Kalman gain
    invterm = np.linalg.inv(np.dot(H, np.dot(P0, np.transpose(H)))+ etavar)
    K = np.dot(P0, np.dot(np.transpose(H), invterm))
    if verbose == True:
        print("K\n", K)
    
    # update the intial belief using the gps measurement
    z = np.array([
        [pgps_i-pc_now],
        [vgps_i-vc_now]
    ])
    residual = z - np.dot(H, dxhat)
    dxhat = dxhat + np.dot(K, residual)
    
    # update Pk
    ikh = np.eye(3) - np.dot(K, H)
    Pk = np.dot(ikh, np.dot(P0, np.transpose(ikh))) + np.dot(K, np.dot(etavar, np.transpose(K)))
    
    # record time-series data for plotting
    dxlist = [dxhat]
    ptruelist = [p_0_true] # true position at each dt_kf
    vtruelist = [v_0_true] # true velocity at each dt_kf
    pclist = [pc_now] # computed position at each dt_kf
    vclist = [vc_now] # computed position at each dt_kf
    Pklist = [np.copy(Pk)]
    Klist = [np.copy(K)]

    
    # run the MC through the time vector
    for i  in range(1, t_acc.shape[0]):
        if i % 40 == 0:
            pc_now, vc_now, ptrue_now, vtrue_now, dxhat, Pk, K = gps_timestep(i, b_a_true, pc_now, vc_now, ptrue_now, vtrue_now, dxhat, Pk, K)
            dxlist.append(dxhat.copy())
            ptruelist.append(ptrue_now)
            vtruelist.append(vtrue_now)
            pclist.append(pc_now)
            vclist.append(vc_now)
            Pklist.append(np.copy(Pk))
            Klist.append(np.copy(K))
        else:
            pc_now, vc_now, ptrue_now, vtrue_now = acc_timestep(i, b_a_true, pc_now, vc_now, ptrue_now, vtrue_now, dxhat[2,0])
    
    return pclist, vclist, ptruelist, vtruelist, b_a_true, dxlist, Pklist, Klist

def gps_timestep(j, ba_true, pc_prev, vc_prev, ptrue_prev, vtrue_prev, dxhat_i, Pk_prev, K):
    # Generates the ith timestep computations of the KF update given both GPS and accelerometer updates
    # i: an int representing the time vec to index into the true acceleration vector, a_truth
    # ba_true: the true bias (a float) for this MC run to be used in the calculation of the noisy accelerometer reading
    # pc_prev: a float representing pc at timestep i-1
    # vc_prev: a float representing vc at timestep i-1
    
    ##### A PRIORI #####
    xbar_next = np.dot(Phi, dxhat_i)
    Mk_next = np.dot(Phi, np.dot(Pk_prev, np.transpose(Phi))) + np.dot(Gamma, np.dot(W, np.transpose(Gamma)))
    
    ##### A POSTERIORI #####
    # update the gain matrix K
    invterm = np.linalg.inv(np.dot(H, np.dot(Mk_next, np.transpose(H)))+ etavar)
    K = np.dot(Mk_next, np.dot(np.transpose(H), invterm))
    
    # calculate the true position and velocity
    # generate an accelerometer reading and computed position & velocity update and true position and velocity
    pc_i, vc_i, ptrue_i, vtrue_i = acc_timestep(j, ba_true, pc_prev, vc_prev, ptrue_prev, vtrue_prev, dxhat_i[2,0])
    
    # generate a noisy GPS measurement for this timestep, i
    eta_i = np.random.normal(etabar, etastd) # noise on the GPS
    pgps_i = ptrue_i + eta_i[0,0]
    vgps_i = vtrue_i + eta_i[1,1]

    # create the measurement vector
    zk = np.array([
        [pgps_i - pc_i],
        [vgps_i - vc_i]
    ])
    
    # calculate the residual
    residual = zk - np.dot(H, xbar_next)
#     print("residual\n", residual)
    
    # update the state estimate
    dxhat = xbar_next + np.dot(K, residual)
    
    # update Pk
#     ikh = np.eye(3) - np.dot(K, H)
#     Pk_new = np.dot(ikh, np.dot(Mk_next, np.transpose(ikh))) + np.dot(K, np.dot(etavar, np.transpose(K)))
    hvh = np.dot(np.transpose(H), np.dot(np.linalg.inv(etavar), H))
    Pk_new = np.linalg.inv(np.linalg.inv(Mk_next) + hvh)
    
    return pc_i, vc_i, ptrue_i, vtrue_i, dxhat, Pk_new, K

def acc_timestep(j, ba_true, pc_prev, vc_prev, ptrue_prev, vtrue_prev, b_est):
    # Generates the ith timestep computations of position and velocity, pc and vc
    # by sampling an accelerometer reading around the true value for this Monte Carlo run
    # i: an int representing the time vec to index into the true acceleration vector, a_truth
    # ba_true: the true bias (a float) for this MC run to be used in the calculation of the noisy accelerometer reading
    # pc_prev: a float representing pc at timestep i-1
    # vc_prev: a float representing vc at timestep i-1
    
    # draw a noise value for the accelerometer
    w_t = np.random.normal(w_mean, w_std)
    
    # gather an accelerometer measurement for this timestep
    ac_prev = a_truth[j-1] + ba_true + w_t #- b_est
    
    # integrate the accelerometer measurment by the euler formula
    vc_i = vc_prev + ac_prev * dt
    pc_i = pc_prev + vc_prev * dt + ac_prev * dt**2/2
    
    # integrate the true acceleration by the euler formula
    vtrue_i = vtrue_prev + a_truth[j-1] * dt
    ptrue_i = ptrue_prev + vtrue_prev * dt + a_truth[j-1] * dt**2/2
    
    return pc_i, vc_i, ptrue_i, vtrue_i
    

In [160]:
pclist, vclist, ptruelist, vtruelist, b_a_true, dxlist, Pklist, Klist = monte_carlo_run(verbose=True)

y = [dxlist[i][2,0] for i in range(len(dxlist))]
dp = [dxlist[i][0,0] for i in range(len(dxlist))]
dv = [dxlist[i][1,0] for i in range(len(dxlist))]
bias_std = [np.sqrt(Pklist[i][2,2]) for i in range(len(Pklist))]
bias_sigma_plus = [y[i] + bias_std[i] for i in range(len(y))]
bias_sigma_minus = [y[i] - bias_std[i] for i in range(len(y))]
pos_error = [ptruelist[i] - pclist[i] for i in range(len(pclist))]
vel_error = [vtruelist[i] - vclist[i] for i in range(len(vclist))]
# Bias plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=t_kf, y=y, name="Estimated bias"))
fig.add_trace(go.Scatter(x=t_kf, y=bias_sigma_plus, name="positive sigma", line=dict(color='firebrick', width=1, dash='dash')))
fig.add_trace(go.Scatter(x=t_kf, y=bias_sigma_minus, name="negative sigma", line=dict(color='firebrick', width=1, dash='dash')))
fig.add_trace(go.Scatter(x=t_kf, y=[b_a_true for i in range(len(dxlist))], name="True bias"))
fig.update_layout(title={'text':"Estimated vs. True Bias",'xanchor':'center','yanchor':'top','y':0.9,'x':0.48}, xaxis_title="Time (sec)", yaxis_title="Bias (m/sec^2)")
fig.show()

# Bias error plot
figb = go.Figure()
figb.add_trace(go.Scatter(x=t_kf, y=[y[i] - b_a_true for i in range(len(dxlist))], name="Bias Error"))
figb.update_layout(title={'text':"Bias Error",'xanchor':'center','yanchor':'top','y':0.9,'x':0.48}, xaxis_title="Time (sec)", yaxis_title="Bias Error (m/sec^2)")
figb.show()


fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=t_kf, y=ptruelist, name="True Position"))
# fig2.add_trace(go.Scatter(x=tlist, y=ptruelist, name="True Position"))
fig2.add_trace(go.Scatter(x=t_kf, y=pclist, name="Computed Position"))
fig2.update_layout(title={'text':"True Position vs. Computed Position",'xanchor':'center','yanchor':'top','y':0.9,'x':0.48}, xaxis_title="Time (sec)", yaxis_title="Position (m)")
fig2.show()

fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=t_kf, y=pos_error, name="Position Error"))
fig3.update_layout(title={'text':"Position Error",'xanchor':'center','yanchor':'top','y':0.9,'x':0.48}, xaxis_title="Time (sec)", yaxis_title="Position Error (m)")
fig3.show()

fig4 = go.Figure()
fig4.add_trace(go.Scatter(x=t_kf, y=vel_error, name="Velocity Error"))
fig4.update_layout(title={'text':"Velocity Error",'xanchor':'center','yanchor':'top','y':0.9,'x':0.48}, xaxis_title="Time (sec)", yaxis_title="Velocity Error (m/s)")
fig4.show()

p0
 9.403104019786099
v0
 98.49127687337042
b0
 -0.1022257517579084
Pk
 [[1.e+02 0.e+00 0.e+00]
 [0.e+00 1.e+00 0.e+00]
 [0.e+00 0.e+00 1.e-02]]
K
 [[0.99009901 0.        ]
 [0.         0.99840256]
 [0.         0.        ]]


In [111]:
Pklist[-1]

array([[ 9.50658743e-03,  2.94831608e-04, -9.88760887e-06],
       [ 2.94831608e-04,  1.62769195e-04, -5.79229156e-06],
       [-9.88760887e-06, -5.79229156e-06,  3.05197532e-06]])

In [138]:
Klist[-1]

array([[ 9.50658743e-03,  1.84269755e-01],
       [ 2.94831608e-04,  1.01730747e-01],
       [-9.88760887e-06, -3.62018222e-03]])

In [ ]:
K
[[ 7.92042100e-03  1.75727322e-01]
 [ 2.81163715e-04  9.48738003e-02]
 [-3.73316458e-08 -1.25969139e-05]]
Mk
[[ 8.03912007e-03  3.13132051e-04 -4.15762569e-08]
 [ 3.13132051e-04  1.67806569e-04 -2.22805969e-08]
 [-4.15762569e-08 -2.22805969e-08  1.06249963e-08]]
Pk
[[ 7.92042100e-03  2.81163715e-04 -3.73316458e-08]
 [ 2.81163715e-04  1.51798080e-04 -2.01550623e-08]
 [-3.73316458e-08 -2.01550623e-08  1.06247141e-08]]